In [15]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from typing import Dict


In [3]:
md = pd.read_csv('data\market_data1.csv')

In [4]:
md.head()

,Time,Instrument,Operation,OrderId,Side,Volume,Price,Lifespan
0,0.0,0,Insert,177349435,B,1000.0,1469.0,G
1,0.0,0,Insert,177349439,A,8000.0,1470.0,G
2,0.0,0,Insert,177349467,A,3000.0,1472.0,G
3,0.0,0,Insert,177349475,A,2000.0,1471.0,G
4,0.0,0,Insert,177349487,A,2000.0,1816.0,G


In [5]:
snapshot = md[md['Time']==(md['Time'].unique()[0])]
snapshot

,Time,Instrument,Operation,OrderId,Side,Volume,Price,Lifespan
0,0.0,0,Insert,177349435,B,1000.0,1469.0,G
1,0.0,0,Insert,177349439,A,8000.0,1470.0,G
2,0.0,0,Insert,177349467,A,3000.0,1472.0,G
3,0.0,0,Insert,177349475,A,2000.0,1471.0,G
4,0.0,0,Insert,177349487,A,2000.0,1816.0,G
...,...,...,...,...,...,...,...,...
96,0.0,0,Insert,177353403,B,2000.0,1130.0,G
97,0.0,0,Cancel,177353403,NaN,NaN,NaN,G
98,0.0,0,Insert,177353407,B,2000.0,1129.0,G
99,0.0,0,Cancel,177353407,NaN,NaN,NaN,G


In [6]:
md[md["Operation"]== "Amend"]

,Time,Instrument,Operation,OrderId,Side,Volume,Price,Lifespan
2318,5.483298,0,Amend,177490955,NaN,-1000.0,NaN,G
8756,17.529436,0,Amend,177827279,NaN,-1000.0,NaN,G
12152,23.291846,0,Amend,177970843,NaN,-20000.0,NaN,G
15706,29.474572,0,Amend,178121187,NaN,-20000.0,NaN,G
16451,31.080238,0,Amend,178354139,NaN,-20000.0,NaN,G
...,...,...,...,...,...,...,...,...
450596,872.495706,0,Amend,204462787,NaN,-20000.0,NaN,G
452096,875.060779,0,Amend,204588539,NaN,-20000.0,NaN,G
454195,880.043257,0,Amend,202608203,NaN,-20000.0,NaN,G
459599,894.573871,0,Amend,205063587,NaN,-20000.0,NaN,G


In [7]:
type(md["OrderId"][1])

numpy.int64

In [17]:
def prodArr(frame):
    volArr = frame['Volume'].values
    priceArr = frame['Price'].values
    prodArr = np.asarray([priceArr[i]*volArr[i] for i in range(len(frame))])
    return prodArr

def bookMaker(asksORbids :pd.DataFrame) -> np.ndarray:
    volArr = asksORbids['Volume'].values
    priceArr = asksORbids['Price'].values
    singleBook = np.empty((len(priceArr),2))
    for i in range(len(priceArr)):
        singleBook[i][0] = volArr[i]
        singleBook[i][1] = priceArr[i]

    return singleBook

# def cancelOrders(frame):





#not categorized by instrument
def orderBookGenerator(df):
    timeshots = df['Time'].unique()
    orderBook = np.empty((len(timeshots),2)) # [[orderBookAsks 0, orderBooksBids 0], ..., [orderBooksAsks n, orderBooksBids n]]
    # orderbook Asks/Bids = [[volume 0, price 0], ..., [volume n, price n]]
    initFrame = df[df['Time']== timeshots[0]].dropna()
    orderBook[0][0] = bookMaker(initFrame[initFrame['Side'] == 'A'])
    orderBook[0][1] = bookMaker(initFrame[initFrame['Side'] == 'B'])



    for snapshotIter in tqdm(range(1, len(timeshots))):
        timeFrame = df[df['Time']==timeshots[snapshotIter]].dropna()


        orderBook[snapshotIter][0] = bookMaker(timeFrame[timeFrame['Side'] == 'A'])
        orderBook[snapshotIter][1] = bookMaker(timeFrame[timeFrame['Side'] == 'B'])

def sideVar(df):
    timeshots = df['Time'].unique()
    timeVarBySide = np.empty((len(timeshots), 2)) # [[varAsks 0, varBids 0 ], ..., [varAsks n, varBids n]]
    for snapIter in tqdm(range(len(timeshots))):

        timeFrame = df[df['Time']==timeshots[snapIter]].dropna()

        asksFrame = timeFrame[timeFrame['Side']=="A"]
        bidsFrame = timeFrame[timeFrame['Side']=='B']

        asksLiquid = prodArr(asksFrame)
        # print(asksLiquid)
        bidsLiquid = prodArr(bidsFrame)

        asksVar = np.var(asksLiquid)
        bidsVar = np.var(bidsLiquid)

        timeVarBySide[snapIter][0] = asksVar
        timeVarBySide[snapIter][1] = bidsVar
    return timeVarBySide



In [18]:
books = pd.read_json("data/books.json").values

FileNotFoundError: File data/books.json does not exist

In [16]:
def getPriceVolArr(book: Dict)-> np.array():
    askVolPriceProd = np.array()
    bidVolPriceProd = np.array()
    for it in range(book["ask_volumes"]):
        askVolPriceProd = np.concatenate(
            (askVolPriceProd,
             [book["ask_prices"][it] for t in book["ask_volumes"][it]]),
            axis=None
        )
    for jit in range(book["bid_volumes"]):
        bidVolPriceProd = np.concatenate(
            (bidVolPriceProd,
            [book["bid_prices"][jit] for t in book["bid_volumes"][it]]),
            axis=None
        )
    return askVolPriceProd, bidVolPriceProd

ask_Var = np.empty(len(books))
bid_Var = np.empty(len(books))
ask_mn = np.empty(len(books))
bid_mn = np.empty(len(books))
for i, book in tqdm(enumerate(books)):
    askVPP, bidVPP = getPriceVolArr(book)
    ask_Var[i] = np.var(askVPP) if len(askVPP) !=0 else 0
    ask_mn[i] = np.mean(askVPP) if len(askVPP) !=0 else 0
    bid_Var[i] = np.var(bidVPP) if len(bidVPP) !=0 else 0
    bid_mn[i] = np.mean(bidVPP) if len(bidVPP) !=0 else 0

TypeError: array() missing required argument 'object' (pos 0)

In [85]:
def getPriceVolArr(book):
    book = book[0]
    askVolPriceProd = np.array([])
    bidVolPriceProd = np.array([])
    

    askVolPriceProd = np.concatenate(
            (askVolPriceProd,
             [book["ask_prices"] for t in book["ask_volumes"]]),
            axis=None
        )
            
    bidVolPriceProd = np.concatenate(
            (bidVolPriceProd,
            [book["bid_prices"] for t in book["bid_volumes"]]),
            axis=None
        )
    return askVolPriceProd, bidVolPriceProd


1
2


In [ ]:
ask_Std = np.empty(len(books))
bid_Std = np.empty(len(books))
ask_mn = np.empty(len(books))
bid_mn = np.empty(len(books))
for i in range(len(books)):
    current_book = books[i]
    askVPP, bidVPP = getPriceVolArr(current_book)
    ask_Std[i] = np.sqrt(np.var(askVPP)) if len(askVPP) !=0 else 0
    ask_mn[i] = np.mean(askVPP) if len(askVPP) !=0 else 0
    bid_Std[i] = np.sqrt(np.var(bidVPP)) if len(bidVPP) !=0 else 0
    bid_mn[i] = np.mean(bidVPP) if len(bidVPP) !=0 else 0
    